In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np
#from scipy.misc import imread

import tensorflow.keras as keras
from tensorflow.keras import Model
from matplotlib.pyplot import imshow
from tensorflow.keras.callbacks import ModelCheckpoint
import cv2

from ultralytics import YOLO
import matplotlib.pyplot as plt
from PIL import Image

from sklearn.metrics import accuracy_score
import pandas as pd

In [16]:
data = keras.utils.image_dataset_from_directory(
    directory='../../data/Faces',
    labels='inferred',
    label_mode='categorical',
    batch_size=140,
    image_size=(1000, 1000),
    class_names = ['Happy', 'Neutral', 'Sad']
)

Found 66 files belonging to 3 classes.


In [17]:
face_classifier = YOLO("../../app/src/face_detection/model/yolov8n-face.pt")

In [18]:
#convert to numpy array
numpy_generator = data.as_numpy_iterator()
X, y = numpy_generator.next()


In [20]:
X.nbytes

num_faces = 0
#create a dataset with cropped faces and its corresponding label
X_cropped = []
y_cropped = []
for i in range(66):
  #  print(i)
    face = np.array(X[i], dtype = 'uint8')
    faces = face_classifier.predict(face)
    boxes = faces[0].boxes.xyxy.tolist()
    if boxes:
        num_faces += 1
        #extract the face based on the output from the YOLOv8 model        
        left, bottom, right, top = boxes[0]
        cropped_face = face[int(bottom):int(top), int(left):int(right)]
        #resize cropped face to a std shape, 100x100 for now but can adjust this
        pil_face = Image.fromarray(np.uint8(cropped_face)).convert('L')
        pil_face = pil_face.resize((48, 48))
        numpy_cropped_face = np.array(pil_face)
        #append this to the new list containing all cropped faces
        X_cropped.append(numpy_cropped_face)
        corresponding_label = y[i]
        #append the labels to the new label list
        y_cropped.append(corresponding_label)
        #uncomment below lines of code to see the cropped face visualisation
        
del X


0: 640x640 1 face, 85.5ms
Speed: 6.0ms preprocess, 85.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 83.2ms
Speed: 3.0ms preprocess, 83.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 76.4ms
Speed: 3.0ms preprocess, 76.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 74.5ms
Speed: 3.0ms preprocess, 74.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 75.0ms
Speed: 3.0ms preprocess, 75.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 81.0ms
Speed: 3.0ms preprocess, 81.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 87.5ms
Speed: 3.0ms preprocess, 87.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 106.5ms
Speed: 4.0ms preprocess, 106.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 64

In [21]:
X_cropped = np.array(X_cropped)
y_cropped = np.array(y_cropped)
X_cropped.shape

(65, 48, 48)

In [25]:
model = keras.models.load_model('trained_emotion_subclasses.keras')

(65, 6)

In [27]:
y_pred = model.predict(X_cropped)



sum(np.argmax(y_pred, axis = 1)==np.argmax(y_cropped, axis = 1))/len(y_pred)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


0.6307692307692307

In [28]:
pd.Series(np.argmax(y_cropped[np.argmax(y_pred, axis = 1)==np.argmax(y_cropped, axis = 1)], axis = 1)).value_counts()

1    20
0    19
2     2
Name: count, dtype: int64